***GENERATED CODE FOR multipletransformation PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***OPERATION FUNCTIONS***

In [ ]:

class Where:
    custom = False
    column1Name = "column1Name"
    column2Name = "column2Name"
    customValue = "customValue"
    valueType = "valueType"
    operator = "operator"
    combinator = "combinator"

    def __init__(self, custom, column1Name, column2Name, customValue, valueType, operator, combinator):
        self.custom = custom
        self.column1Name = column1Name
        self.column2Name = column2Name
        self.customValue = customValue
        self.valueType = valueType
        self.operator = operator
        self.combinator = combinator


class FilterOperation:

    def __init__(self):
        pass

    @staticmethod
    def parse(attributes):
        filters = []
        for filter in attributes["expression"]:
            filters.append(Where(filter["custom"],
                                 filter["column1_name"] or None,
                                 filter["column2_name"] or None,
                                 filter["custom_value"] or None,
                                 filter["value_type"] or None,
                                 filter["operator"] or None,
                                 filter["combinator"] or ""))
        return filters

    def run(inStages, inStagesData, stageId, sparkSession, stageAttribs):
        filters = FilterOperation.parse(eval(stageAttribs))
        whereClause = ""
        for f in filters:
            if f.custom:
                customValue = f.customValue
                if f.valueType.lower() == "string":
                    customValue = "'" + customValue + "'"
                whereClause += "%s %s %s %s " % (f.combinator,
                                                 f.column1Name, f.operator, customValue)
            else:
                whereClause += "%s %s %s %s " % (f.combinator,
                                                 f.column1Name, f.operator, f.column2Name)

        inStagesData[inStages[0]].createOrReplaceTempView("view%s" % (stageId))
        query = "SELECT * FROM %s WHERE %s" % ("view%s" %
                                               (stageId), whereClause)
        return sparkSession.sql(query)


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
from dask.dataframe import from_pandas
import json


def replaceValues(df, functionsData, applyOn):
    for columnData in applyOn:
        for functionData in functionsData:
            if functionData['replaceType'] == 'value':
                if functionData['replaceValueType'] == 'min':
                    minValue = df[columnData['columnName']].min().compute()
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, minValue)
                elif functionData['replaceValueType'] == 'max':
                    maxValue = df[columnData['columnName']].max().compute()
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, maxValue)
                else:
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, functionData['ReplaceWith'])
            elif functionData['replaceType'] == 'none':
                if functionData['replaceValueType'] == 'min':
                    minValue = df[columnData['columnName']].min().compute()
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].replace("", minValue)
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].fillna(minValue)
                elif functionData['replaceValueType'] == 'max':
                    maxValue = df[columnData['columnName']].max().compute()
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].replace("", maxValue)
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].fillna(maxValue)
                else:
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        "", functionData['ReplaceWith'])
                    df[columnData['columnName']] = df[columnData['columnName']].fillna(
                        functionData['ReplaceWith'])
    return df


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'State', 'transformation_label': 'String Indexer'}], 'feature': 'State', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '3332', 'mean': '', 'stddev': '', 'min': 'AK', 'max': 'WY', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'State'}, {'feature_label': 'State', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('State')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Phone', 'transformation_label': 'String Indexer'}], 'feature': 'Phone', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3332', 'mean': '', 'stddev': '', 'min': '327-1058', 'max': '422-9964', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Phone'}, {'feature_label': 'Phone', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Phone')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Intl_Plan', 'transformation_label': 'String Indexer'}], 'feature': 'Intl_Plan', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3332', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Intl_Plan'}, {'feature_label': 'Intl_Plan', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Intl_Plan')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'VMail_Plan', 'transformation_label': 'String Indexer'}], 'feature': 'VMail_Plan', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3332', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'VMail_Plan'}, {'feature_label': 'VMail_Plan', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('VMail_Plan')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'cluster_labels', 'transformation_label': 'String Indexer'}], 'feature': 'cluster_labels', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3332', 'mean': '', 'stddev': '', 'min': 'day_callers', 'max': 'vmailers', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'cluster_labels'}, {'feature_label': 'cluster_labels', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('cluster_labels')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run multipletransformationHooks.ipynb
try:
	#sourcePreExecutionHook()

	predictivechurntrain = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/Data/TrainData/PredictiveChurnTrain.csv', 'filename': 'PredictiveChurnTrain.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(predictivechurntrain)

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run multipletransformationHooks.ipynb
try:
	filter = FilterOperation.run(["60cc86c0b1e28e47f2c45be1"],{"60cc86c0b1e28e47f2c45be1": predictivechurntrain}, "60cc86c7b1e28e47f2c45be2", spark, "{'url': '/FileStore/platform/Data/TrainData/PredictiveChurnTrain.csv', 'expression': [{'column1_name': 'Intl_Plan', 'operator': '=', 'column2_name': 'VMail_Plan', 'custom': False, 'custom_value': '', 'value_type': '', 'combinator': '', 'showOptions': True}], 'use_expression': True, 'regex': [], 'use_regex': False, 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run multipletransformationHooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(filter,spark,json.dumps( {"url": "/FileStore/platform/Data/TrainData/PredictiveChurnTrain.csv", "source_attributes": {}, "DataPrepFile": "/FileStore/platform/Data/TrainData/PredictiveChurnTrain.csv", "data_source": "HDFS", "startListenerOnly": 1, "dateColumnNames": [], "FilePath": "/FileStore/platform/extra/60cc86cdb1e28e47f2c45be31624017132/0part.csv", "requestRatio": 7.0, "totalRows": 3332, "BasicStats": {"missingValues": 0.0, "numberOfColumns": 24, "numberOfRows": 3332, "duplicateRowCount": 0, "stats": [{"column": "State", "alias": "State", "generated": 0, "type": "String", "max": "WY", "min": "AK", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "Account_Length", "alias": "Account_Length", "generated": 0, "type": "real", "max": 243.0, "min": 1.0, "mean": 101.05672268907563, "missing": 0.0, "stddev": 39.83}, {"column": "Area_Code", "alias": "Area_Code", "generated": 0, "type": "real", "max": 510.0, "min": 408.0, "mean": 437.1890756302521, "missing": 0.0, "stddev": 42.38}, {"column": "Phone", "alias": "Phone", "generated": 0, "type": "String", "max": "422-9964", "min": "327-1058", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "Intl_Plan", "alias": "Intl_Plan", "generated": 0, "type": "String", "max": "yes", "min": "no", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "VMail_Plan", "alias": "VMail_Plan", "generated": 0, "type": "String", "max": "yes", "min": "no", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "VMail_Message", "alias": "VMail_Message", "generated": 0, "type": "real", "max": 51.0, "min": 0.0, "mean": 8.093937575030012, "missing": 0.0, "stddev": 13.69}, {"column": "Day_Mins", "alias": "Day_Mins", "generated": 0, "type": "real", "max": 350.8, "min": 0.0, "mean": 179.74948979591855, "missing": 0.0, "stddev": 54.46}, {"column": "Day_Calls", "alias": "Day_Calls", "generated": 0, "type": "real", "max": 165.0, "min": 0.0, "mean": 100.4327731092437, "missing": 0.0, "stddev": 20.07}, {"column": "Day_Charge", "alias": "Day_Charge", "generated": 0, "type": "real", "max": 59.64, "min": 0.0, "mean": 30.557953181272534, "missing": 0.0, "stddev": 9.26}, {"column": "Eve_Mins", "alias": "Eve_Mins", "generated": 0, "type": "real", "max": 363.7, "min": 0.0, "mean": 200.98142256902753, "missing": 0.0, "stddev": 50.72}, {"column": "Eve_Calls", "alias": "Eve_Calls", "generated": 0, "type": "real", "max": 170.0, "min": 0.0, "mean": 100.11464585834334, "missing": 0.0, "stddev": 19.93}, {"column": "Eve_Charge", "alias": "Eve_Charge", "generated": 0, "type": "real", "max": 30.91, "min": 0.0, "mean": 17.083631452581024, "missing": 0.0, "stddev": 4.31}, {"column": "Night_Mins", "alias": "Night_Mins", "generated": 0, "type": "real", "max": 395.0, "min": 23.2, "mean": 200.8588835534214, "missing": 0.0, "stddev": 50.58}, {"column": "Night_Calls", "alias": "Night_Calls", "generated": 0, "type": "real", "max": 175.0, "min": 33.0, "mean": 100.11044417767107, "missing": 0.0, "stddev": 19.57}, {"column": "Night_Charge", "alias": "Night_Charge", "generated": 0, "type": "real", "max": 17.77, "min": 1.04, "mean": 9.038733493397373, "missing": 0.0, "stddev": 2.28}, {"column": "Intl_Mins", "alias": "Intl_Mins", "generated": 0, "type": "real", "max": 20.0, "min": 0.0, "mean": 10.237364945978387, "missing": 0.0, "stddev": 2.79}, {"column": "total_Mins", "alias": "total_Mins", "generated": 0, "type": "real", "max": 885.0, "min": 284.3, "mean": 591.8271608643462, "missing": 0.0, "stddev": 89.94}, {"column": "Intl_Calls", "alias": "Intl_Calls", "generated": 0, "type": "real", "max": 20.0, "min": 0.0, "mean": 4.479891956782713, "missing": 0.0, "stddev": 2.46}, {"column": "Intl_Charge", "alias": "Intl_Charge", "generated": 0, "type": "real", "max": 5.4, "min": 0.0, "mean": 2.764600840336124, "missing": 0.0, "stddev": 0.75}, {"column": "Total_Charge", "alias": "Total_Charge", "generated": 0, "type": "real", "max": 96.15, "min": 22.93, "mean": 59.444918967586965, "missing": 0.0, "stddev": 10.5}, {"column": "CustServ_Calls", "alias": "CustServ_Calls", "generated": 0, "type": "real", "max": 9.0, "min": 0.0, "mean": 1.5630252100840336, "missing": 0.0, "stddev": 1.32}, {"column": "Churn", "alias": "Churn", "generated": 0, "type": "real", "max": 1.0, "min": 0.0, "mean": 0.14495798319327732, "missing": 0.0, "stddev": 0.35}, {"column": "cluster_labels", "alias": "cluster_labels", "generated": 0, "type": "String", "max": "vmailers", "min": "day_callers", "mean": "", "missing": 0.0, "stddev": ""}]}, "HasBasicStats": 1, "functionsApplied": [{"functionName": "replaceBy", "applyOn": [{"columnName": "Intl_Plan", "type": "String", "min": "no", "max": "yes", "mean": "-"}], "functionsData": [{"replaceType": "none", "toReplace": "", "asNewColumn": 0, "newColumnName": "", "replaceValueType": "max", "ReplaceWith": ""}]}], "functionChanges": [{"columnName": "Intl_Plan", "functionName": "Replace Outliers", "Type": "String", "Parameters": [{"replaceType": "none", "toReplace": "", "asNewColumn": 0, "newColumnName": "", "replaceValueType": "max", "ReplaceWith": ""}]}], "fileheader": [{"field": "State", "alias": "State", "generated": 0, "position": 1, "type": "String"}, {"field": "Account_Length", "alias": "Account_Length", "generated": 0, "position": 2, "type": "real"}, {"field": "Area_Code", "alias": "Area_Code", "generated": 0, "position": 3, "type": "real"}, {"field": "Phone", "alias": "Phone", "generated": 0, "position": 4, "type": "String"}, {"field": "Intl_Plan", "alias": "Intl_Plan", "generated": 0, "position": 5, "type": "String"}, {"field": "VMail_Plan", "alias": "VMail_Plan", "generated": 0, "position": 6, "type": "String"}, {"field": "VMail_Message", "alias": "VMail_Message", "generated": 0, "position": 7, "type": "real"}, {"field": "Day_Mins", "alias": "Day_Mins", "generated": 0, "position": 8, "type": "real"}, {"field": "Day_Calls", "alias": "Day_Calls", "generated": 0, "position": 9, "type": "real"}, {"field": "Day_Charge", "alias": "Day_Charge", "generated": 0, "position": 10, "type": "real"}, {"field": "Eve_Mins", "alias": "Eve_Mins", "generated": 0, "position": 11, "type": "real"}, {"field": "Eve_Calls", "alias": "Eve_Calls", "generated": 0, "position": 12, "type": "real"}, {"field": "Eve_Charge", "alias": "Eve_Charge", "generated": 0, "position": 13, "type": "real"}, {"field": "Night_Mins", "alias": "Night_Mins", "generated": 0, "position": 14, "type": "real"}, {"field": "Night_Calls", "alias": "Night_Calls", "generated": 0, "position": 15, "type": "real"}, {"field": "Night_Charge", "alias": "Night_Charge", "generated": 0, "position": 16, "type": "real"}, {"field": "Intl_Mins", "alias": "Intl_Mins", "generated": 0, "position": 17, "type": "real"}, {"field": "total_Mins", "alias": "total_Mins", "generated": 0, "position": 18, "type": "real"}, {"field": "Intl_Calls", "alias": "Intl_Calls", "generated": 0, "position": 19, "type": "real"}, {"field": "Intl_Charge", "alias": "Intl_Charge", "generated": 0, "position": 20, "type": "real"}, {"field": "Total_Charge", "alias": "Total_Charge", "generated": 0, "position": 21, "type": "real"}, {"field": "CustServ_Calls", "alias": "CustServ_Calls", "generated": 0, "position": 22, "type": "real"}, {"field": "Churn", "alias": "Churn", "generated": 0, "position": 23, "type": "real"}, {"field": "cluster_labels", "alias": "cluster_labels", "generated": 0, "position": 24, "type": "String"}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run multipletransformationHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(datapreparation,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "State", "transformation_label": "String Indexer"}], "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3332", "mean": "", "stddev": "", "min": "AK", "max": "WY", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "State"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Account_Length", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "101.06", "stddev": "39.83", "min": "1.0", "max": "243.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Account_Length"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Area_Code", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "437.19", "stddev": "42.38", "min": "408.0", "max": "510.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Area_Code"}, {"transformationsData": [{"feature_label": "Phone", "transformation_label": "String Indexer"}], "feature": "Phone", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3332", "mean": "", "stddev": "", "min": "327-1058", "max": "422-9964", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Phone"}, {"transformationsData": [{"feature_label": "Intl_Plan", "transformation_label": "String Indexer"}], "feature": "Intl_Plan", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3332", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Intl_Plan"}, {"transformationsData": [{"feature_label": "VMail_Plan", "transformation_label": "String Indexer"}], "feature": "VMail_Plan", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3332", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "VMail_Plan"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "VMail_Message", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "8.09", "stddev": "13.69", "min": "0.0", "max": "51.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "VMail_Message"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Day_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "179.75", "stddev": "54.46", "min": "0.0", "max": "350.8", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Day_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Day_Calls", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "100.43", "stddev": "20.07", "min": "0.0", "max": "165.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Day_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Day_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "30.56", "stddev": "9.26", "min": "0.0", "max": "59.64", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Day_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Eve_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "200.98", "stddev": "50.72", "min": "0.0", "max": "363.7", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Eve_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Eve_Calls", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "100.11", "stddev": "19.93", "min": "0.0", "max": "170.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Eve_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Eve_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "17.08", "stddev": "4.31", "min": "0.0", "max": "30.91", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Eve_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Night_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "200.86", "stddev": "50.58", "min": "23.2", "max": "395.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Night_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Night_Calls", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "100.11", "stddev": "19.57", "min": "33.0", "max": "175.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Night_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Night_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "9.04", "stddev": "2.28", "min": "1.04", "max": "17.77", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Night_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intl_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "10.24", "stddev": "2.79", "min": "0.0", "max": "20.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intl_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "591.83", "stddev": "89.94", "min": "284.3", "max": "885.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intl_Calls", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "4.48", "stddev": "2.46", "min": "0.0", "max": "20.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intl_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intl_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "2.76", "stddev": "0.75", "min": "0.0", "max": "5.4", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intl_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Total_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "59.44", "stddev": "10.5", "min": "22.93", "max": "96.15", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Total_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustServ_Calls", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "1.56", "stddev": "1.32", "min": "0.0", "max": "9.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CustServ_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Churn", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "0.14", "stddev": "0.35", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Churn"}, {"transformationsData": [{"feature_label": "cluster_labels", "transformation_label": "String Indexer"}], "feature": "cluster_labels", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3332", "mean": "", "stddev": "", "min": "day_callers", "max": "vmailers", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "cluster_labels"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run multipletransformationHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["State_stringindexer", "Account_Length", "Area_Code", "Phone_stringindexer", "Intl_Plan_stringindexer", "VMail_Plan_stringindexer", "VMail_Message", "Day_Mins", "Day_Calls", "Day_Charge", "Eve_Mins", "Eve_Calls", "Eve_Charge", "Night_Mins", "Night_Calls", "Night_Charge", "Intl_Mins", "total_Mins", "Intl_Calls", "Intl_Charge", "Total_Charge", "CustServ_Calls", "cluster_labels_stringindexer"], "Churn")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

